In [1]:
import json
import random
import gensim

from gensim.parsing.preprocessing import preprocess_documents

### Upload dei file
La variabile **corpus** è una lista delle:
- frasi contenute negli articoli 
- domande delle FAQ

In [2]:
corpus = list()

In [3]:
with open ('articoli.txt', 'r', encoding = 'utf-8') as file:
    corpus = json.loads(file.read())
    
# eliminazione di quelle frasi che sono delle domande
# (per evitare che il chatbot risponda con delle domande)
for sentence in corpus:
    if (sentence[len(sentence)-1] == '?'):
        corpus.remove(sentence)

**q_a** dizionario che ha per chiave le domande delle FAQ e per valore una tupla contenente la risposta alla FAQ e il link della pagina web in cui le FAQ sono state estratte

In [4]:
with open('q&a.json', 'r') as data_structure:
    q_a = json.load(data_structure)
    # appendiamo le chiavi del dizionario (le domande delle FAQ)
    for k in q_a.keys():
        corpus.append(k)

### Pre-processing
Per il preprocessing abbiamo usato una funzione di gensim **simple_preprocess** che:
- prende in input un documento (nel nostro caso una frase)
- trasforma tutto in lower case
- tokenizza
- restituisce una lista di tokens

In [5]:
# funzione che effettua il preprocessing di ogni frase contenuta 
# nel corpus
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))

In [6]:
# richiamiamo la funzione 
processed_corpus =list(sent_to_words(corpus))

### Costruzione del modello

Creazione del dizionario e rappresentazione Bag Of Words
- corpora.Dictionary = serve per creare il concetto di dizionario (inteso come mapping tra una parola e un suo identificatore numerico)

In [7]:
dictionary = gensim.corpora.Dictionary(processed_corpus)

# forniamo a ogni documento/frase del corpus una rappresentazione bow
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

**bow_corpus** è una lista di frasi in cui ogni parola è rappresentata da una tupla: il primo elemento della tupla è l'id di quella parola nel dizionario, il secondo elemento è un numero che indica quante volte quella parola appare nella frase

In [8]:
# carichiamo il modello tfidf dalla gensim library
tfidf = gensim.models.TfidfModel(bow_corpus, smartirs='npu')
# applichiamo il modello alla precedente rappresentazione bow
corpus_tfidf = tfidf[bow_corpus]

**LSI = Latent Semantic Indexing**

Si tratta di un algoritmo che restituisce come output un vettore a bassa dimensionalità che rappresenta il documento/frase. Nel nostro caso, l'input è un insieme di vettori che hanno una rappresentazione tf-idf.

- num_topics = number of latent dimensions

In [9]:
lsi = gensim.models.LsiModel(corpus_tfidf, num_topics=200)

**MatrixSimilarity** = funzione di gensim che calcola la cosine similarity tra i documenti di un corpus. La matrice di similarità viene indicizzata. Prende in input vettori densi (per questo gli si passa l'output di LSI).

In [10]:
index = gensim.similarities.MatrixSimilarity(lsi[corpus_tfidf])

### Impostazione chatbot

In [11]:
# funzione che restituisce un saluto come risposta a un saluto dell'utente
def greeting_response(text):
    text = text.lower()
    
    # lista dei possibili saluti che il bot può dare
    bot_greetings = ['ciao', 'ehi']
    # lista dei saluti che il bot riconosce
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [12]:
def bot_response(user_input, index, q_a):
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    found_faq = False
    
    # minimo preprocessing dell'input
    processed_input = user_input.lower().split()
    bow_query_vec = dictionary.doc2bow(processed_input)
    tfidf_query_vec = tfidf[bow_query_vec]
    
    # facciamo rappresentazione densa del vettore query
    lsi_query_vec = lsi[tfidf_query_vec]
    
    sims = index[lsi_query_vec]
    sims = sorted(enumerate(sims), key=lambda item: item[1], reverse=True)
    
    # se la frase più simile che ha trovato è una domanda delle FAQ
    # restituisco la risposta
    for k in q_a.keys():
        if corpus[sims[0][0]].lower() == k.lower():
            bot_response = bot_response + q_a[k][0]
            response_flag = 1 # abbiamo trovato la risposta
            found_faq = True # abbiamo trovato la risposta

    if not found_faq:
        bot_response = bot_response + corpus[sims[0][0]]
        response_flag = 1 # abbiamo trovato la risposta
    
    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'
        
    return bot_response

### Esempio di conversazione (try it yourself!)

In [13]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
while(True):
    print()
    print('User:')
    user_input = input()
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, index, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:


KeyboardInterrupt: Interrupted by user

### Testing
Il nostro test set è una lista di 42 domande che sono state raccolte con le modalità descritte nel report.

In [14]:
test_set = ["Cosa fare nel caso in cui si manifestano i sintomi?", 
            "Posso viaggiare se ho il coronavirus?", 
            "Le donne in gravidanza possono trasmettere la malattia al figlio?", 
            "Quanto tempo resta nel corpo il coronavirus?", 
            "Come funziona il contact tracing?", 
            "Quando finirà la pandemia?", 
            "Il vaccino ha effetti collaterali?", 
            "Cosa sappiamo della nuova variante inglese?", 
            "Quanti tipi di vaccino esistono?", 
            "Fino a quando saranno chiuse le attività commerciali?", 
            "Come distinguiamo un'influenza stagionale da un'influenza covid?", 
            "Chi si è vaccinato può trasmette la malattia?", 
            "Quali sono i colori che possono assumere le regioni?", 
            "Come fanno le persone infette a trasmettere il virus?", 
            "Se vengo dall'estero devo fare il tampone?", 
            "Cos’è l'indice Rt?", 
            "Posso ammalarmi due volte di covid?", 
            "Quanto tempo dura l'isolamento fiduciario?", 
            "È possibile fare sport di contatto?", 
            "Come faccio a capire se sono positiva ma asintomatica?", 
            "Sono stato a contatto con un infetto: cosa devo fare?", 
            "Fino a quando resteranno chiuse le piscine e le palestre?", 
            "Dopo il vaccino, devo comunque continuare a usare la mascherina?", 
            "Che cosa devo fare se voglio andare all'estero?", 
            "Lo smog/inquinamento influisce sulla diffusione del coronavirus/covid?", 
            "Qual è il periodo di copertura del vaccino?", 
            "Se sono all’estero, cosa devo fare per tornare in Italia?", 
            "Come proteggere le persone più anziane dal virus?", 
            "Il coronavirus è pericoloso per gli animali domestici?", 
            "La mascherina è obbligatoria all’aperto?", 
            "Fino a quando sarà adottata la didattica a distanza per le scuole superiori?", 
            "Qual è l'incidenza del covid nei bambini?", 
            "I guanti servono a proteggere dal contagio?", 
            "Si può viaggiare fuori comune in area gialla?", 
            "Quanto sono importanti il distanziamento e la chiusura delle attività nella trasmissione del covid?", 
            "Posso fare attività sportiva se sono positivo asintomatico?", 
            "Per i bambini è obbligatorio l’uso della mascherina?", 
            "Come agiscono i linfociti T nelle infezioni da covid?", 
            "Posso visitare i miei nonni?", 
            "È possibile andare a pesca?", 
            "Quali sono i sintomi del coronavirus?", 
            "Il virus può rimanere sulle superfici?"]

In [15]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

for sentence in test_set:
    user_input = sentence
    
    print()
    print('User:')
    print (user_input)

    print('Co-bot: ... ')
    print(bot_response(user_input, index, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:
Cosa fare nel caso in cui si manifestano i sintomi?
Co-bot: ... 
Il referente scolastico per il Covid-19 deve comunicare al Dipartimento di Prevenzione se si verifica un numero elevato di assenze improvvise di studenti in una classe o di insegnanti. Il Dipartimento effettuerà un’indagine epidemiologica per valutare le azioni di sanità pubblica da intraprendere, tenendo conto della presenza di casi confermati nella scuola o di focolai di Covid-19 nella comunità.

User:
Posso viaggiare se ho il coronavirus?
Co-bot: ... 
L'influenza e il virus che causa COVID-19 sono due virus diversi e il vaccino contro l'influenza stagionale non protegge da COVID-19. La vaccinazione anti-influenzale è fortemente raccomandata perché rende la diagnosi differenziale (cioè la distinzione tra le due infezioni), più facile e più rapida, portando più precocemente all'isolamento di eventuali casi di coronavirus.

User:
Le donne i

Il nuovo DPCM dispone per le scuole secondarie il ricorso alla didattica digitale integrata, che deve comunque rimanere complementare alla didattica in presenza, modulando ulteriormente la gestione degli orari di ingresso e di uscita degli alunni, anche attraverso l’eventuale utilizzo di turni pomeridiani e disponendo che l’ingresso non avvenga in ogni caso prima delle 9.00.Le università invece dovranno essere in grado di adattarsi in base all'andamento del quadro epidemiologico, prevedendo la possibilità di didattica a distanza in caso di necessità.

User:
Qual è l'incidenza del covid nei bambini?
Co-bot: ... 
Allo stato attuale non esistono evidenze scientifiche che il vaccino contro la tubercolosi (TBC), possa fornire protezione contro il nuovo coronavirus (SARS-CoV-2). Questo vaccino resta indicato in particolari categorie di popolazione a rischio per queste infezioni.

User:
I guanti servono a proteggere dal contagio?
Co-bot: ... 
Al momento non esistono evidenze che gli antiretro